In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [6]:
from TileEM_plot_toolbox import *
from qualityBaseline import *
from glob import glob

worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}
sampleN_lst=worker_Nbatches.keys()

from collections import OrderedDict
df = pd.read_csv("../computed_my_COCO_BBvals.csv",index_col=0)

In [29]:
cd ../../

/home/jlee782/crowd-seg/analysis/TileEM


In [39]:
attr = "Area Ratio"
for Nworker in tqdm(sampleN_lst):
    for batch_num in range(worker_Nbatches[Nworker]):
        dir_name = "{0}worker_rand{1}".format(Nworker,batch_num)
        #print dir_name
        os.chdir("sample/"+dir_name)
         
        # Creating 2 empty precision recall table containing P,R for each metric used 
        cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']

        PR_tbl = pd.DataFrame()#index=object_lst)#,columns=cols)
        PR_tbl["object_id"]=object_lst
        # Summarization-Based Method
        df.ix[df["Precision [Self]"]>1,"Precision [Self]"]=1
        df.ix[df["Recall [Self]"]>1,"Recall [Self]"]=1
        df = df[~(df["worker_id"].isin([1,2,3]))] #Filter our ground truth workers
#         for attr in cols:
        tbl=[]
        for objid in object_lst:
            workers=pkl.load(open("worker{}.pkl".format(objid)))
            filtered_df = df[(df["worker_id"].isin(workers))&(df["object_id"]==objid)] #only look at summarization scores of sampled workers
            best_worker_BB = filtered_df[filtered_df[attr]==filtered_df[attr].max()]
            if objid not in [35,41]:
                tbl.append([objid,best_worker_BB["x_locs"].values[0],best_worker_BB["y_locs"].values[0],best_worker_BB[attr].values[0]])
            else:
                tbl.append([objid,-1,-1])
            #tbl.append([objid,best_worker_BB["Precision [Self]"].values[0],best_worker_BB["Recall [Self]"].values[0]])
        tmp_PR_tbl = pd.DataFrame(tbl,columns=["object_id","x_locs","y_locs",attr])
        PR_tbl["x_locs".format(attr)]=tmp_PR_tbl["x_locs"]
        PR_tbl["y_locs".format(attr)]=tmp_PR_tbl["y_locs"]
        PR_tbl[attr]=tmp_PR_tbl[attr]
        PR_tbl.to_csv("best_{}_BBs.csv".format(attr.replace(" ","_")))
        os.chdir("../..")
#         # Vision based methods 
#         for threshold in [10,50,90]:
#             visionPR = pd.read_csv("../../../PR{}.csv".format(threshold))
#             PR_tbl["P [Vision{}%]".format(threshold)] = visionPR["precision"]
#             PR_tbl["R [Vision{}%]".format(threshold)] = visionPR["recall"]
#         #MVT, Tile
#         tbl=[]
#         for fname in glob("Tstar_idx_obj*.pkl"):
#             objid=int(fname.split("_")[-1].split(".")[0][3:])
#             tiles = pkl.load(open("vtiles{}.pkl".format(objid)))
#             #Tile EM
#             Tstar_lst = pkl.load(open("Tstar_idx_obj{}.pkl".format(objid)))
#             TileEMP,TileEMR=compute_PR(objid,np.array(Tstar_lst[-1]),tiles)
#             # Majority Vote 
#             PMVT,RMVT = majority_vote(objid,heuristic="50%")
#             PMVTtopk,RMVTtopk = majority_vote(objid,heuristic="topk")
#             PMVTtopP,RMVTtopP = majority_vote(objid,heuristic="topPercentile")
#             tbl.append([objid,TileEMP,TileEMR,PMVT,RMVT,PMVTtopk,RMVTtopk,PMVTtopP,RMVTtopP])
#         Tile_df = pd.DataFrame(tbl,columns=["object_id","P [TileEM]","R [TileEM]","P [MVT]","R [MVT]","P [MVTtop10]","R [MVTtop10]","P [MVTtop95%]","R [MVTtop95%]"])
#         PR_tbl_all = PR_tbl.merge(Tile_df,on="object_id")
#         #Save to file in that folder 
#         PR_tbl_all.to_csv("PR_tbl_all.csv")
        

In [42]:
cat sample/10worker_rand0/best_Area_Ratio_BBs.csv

,object_id,x_locs,y_locs,Area Ratio
0,1,"[330, 308, 300, 300, 306, 308, 288, 283, 272, 265, 240, 212, 198, 183, 190, 219, 236, 253, 264, 259, 271, 258, 259, 268, 271, 288, 302, 324, 344, 357, 366, 378, 389, 398, 408, 414, 419, 421, 418, 410, 398, 386, 376, 372, 369, 370, 365, 363, 356, 345]","[151, 164, 186, 198, 213, 227, 241, 248, 268, 278, 294, 302, 311, 317, 332, 324, 326, 323, 321, 335, 339, 347, 360, 368, 376, 373, 369, 369, 383, 363, 348, 342, 339, 338, 336, 311, 301, 288, 270, 255, 241, 232, 224, 217, 199, 187, 174, 166, 157, 151]",0.0835026041667
1,2,"[225, 251, 242, 233, 224, 224, 222, 216, 206, 214, 207, 202, 195, 192, 191, 183, 183]","[428, 415, 336, 324, 305, 299, 290, 287, 281, 316, 297, 293, 295, 303, 327, 330, 411]",0.0231363932292
2,3,"[47, 65, 81, 89, 93, 92, 90, 88, 85, 71, 55, 47]","[317, 326, 331, 343, 356, 369, 388, 410, 418, 418, 410, 400]",0.0121695963542
3,4,"[88, 92, 88, 86, 83, 83, 82, 83, 102, 116, 133, 123, 117, 117, 107]","[336, 354, 371, 387, 402, 413, 42

In [41]:
cat sample/10worker_rand0/best_Num_Points_BBs.csv

,object_id,x_locs,y_locs,Num Points
0,1,"[299, 300, 302, 307, 312, 321, 335, 347, 357, 362, 366, 363, 359, 361, 370, 375, 385, 400, 408, 414, 417, 420, 418, 415, 413, 410, 409, 400, 382, 369, 361, 351, 348, 342, 334, 322, 303, 297, 286, 279, 272, 260, 261, 264, 258, 271, 278, 268, 264, 256, 248, 242, 235, 224, 218, 208, 198, 191, 185, 179, 185, 192, 205, 223, 233, 243, 249, 255, 262, 269, 274, 278, 283, 290, 295, 299, 306, 306, 301, 300, 293]","[190, 175, 168, 161, 155, 149, 147, 150, 157, 168, 185, 195, 202, 209, 215, 220, 230, 241, 249, 256, 266, 281, 293, 300, 307, 323, 331, 332, 335, 343, 355, 372, 382, 380, 376, 369, 368, 368, 369, 371, 371, 359, 352, 346, 340, 338, 336, 330, 322, 325, 325, 323, 322, 320, 322, 323, 325, 325, 322, 319, 316, 309, 304, 301, 297, 291, 291, 284, 280, 268, 261, 254, 247, 238, 232, 226, 225, 214, 209, 201, 195]",81.0
1,2,"[186, 185, 185, 185, 193, 196, 202, 208, 221, 227, 233, 238, 242, 243, 246, 247, 248, 242, 231, 225, 208, 194]","[412, 378, 350, 333, 